Chapter 24

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Now that we have described some of the core pieces you can expect to come across, let’s create a
simple pipeline to demonstrate each of the components. We’ll use a small synthetic dataset that
will help illustrate our point. Let’s read the data in and see a sample before talking about it
further:

In [2]:
df = spark.read.json("./../data/simple-ml")
df.orderBy("value2").show()

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
|green| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|green|good|     1|14.386294994851129|
|green|good|    12|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|  red|good|    35|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



This dataset consists of a categorical label with two values (good or bad), a categorical variable
(color), and two numerical variables. While the data is synthetic, let’s imagine that this dataset
represents a company’s customer health. The “color” column represents some categorical health
rating made by a customer service representative. The “lab” column represents the true customer
health. The other two values are some numerical measures of activity within an application (e.g.,
minutes spent on site and purchases). Suppose that we want to train a classification model where
we hope to predict a binary variable—the label—from the other values

Feature engineering with transformers

As already mentioned, transformers help us manipulate our current columns in one way or
another. Manipulating these columns is often in pursuit of building features (that we will input
into our model). Transformers exist to either cut down the number of features, add more features,
manipulate current ones, or simply to help us format our data correctly. Transformers add new
columns to DataFrames.
When we use MLlib, all inputs to machine learning algorithms (with several exceptions
discussed in later chapters) in Spark must consist of type Double (for labels) and
Vector[Double] (for features). The current dataset does not meet that requirement and therefore
we need to transform it to the proper format.
To achieve this in our example, we are going to specify an RFormula. This is a declarative
language for specifying machine learning transformations and is simple to use once you
understand the syntax. RFormula supports a limited subset of the R operators that in practice
work quite well for simple models and manipulations (we demonstrate the manual approach to
this problem in Chapter 25). The basic RFormula operators are:

In order to specify transformations with this syntax, we need to import the relevant class. Then
we go through the process of defining our formula. In this case we want to use all available
variables (the .) and also add in the interactions between value1 and color and value2 and
color, treating those as new features:

~
Separate target and terms
+
Concat terms; “+ 0” means removing the intercept (this means that the y-intercept of the line
that we will fit will be 0)
-
Remove a term; “- 1” means removing the intercept (this means that the y-intercept of the
line that we will fit will be 0—yes, this does the same thing as “+ 0”
:
Interaction (multiplication for numeric values, or binarized categorical values)
.
All columns except the target/dependent variable

In [3]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")

At this point, we have declaratively specified how we would like to change our data into what we
will train our model on. The next step is to fit the RFormula transformer to the data to let it
discover the possible values of each column. Not all transformers have this requirement but
because RFormula will automatically handle categorical variables for us, it needs to determine
which columns are categorical and which are not, as well as what the distinct values of the
categorical columns are. For this reason, we have to call the fit method. Once we call fit, it
returns a “trained” version of our transformer we can then use to actually transform our data.

In [4]:
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show()

+-----+----+------+------------------+--------------------+-----+
|color| lab|value1|            value2|            features|label|
+-----+----+------+------------------+--------------------+-----+
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6,9],[12...|  0.0|
|green|good|    15| 38.97187133755819|(10,[1,2,3,5,8],[...|  1.0|
|green|good|    12|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
|green| bad|    16|14.386294994851129|(10,[1,2,3,5,8],[...|  0.0|
|  red|good|    35|14.386294994851129|(10,[0,2,3,4,7],[...|  1.0|
|  red| bad|     1| 38.97187133755819|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|     2|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|    16|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red|good|    45| 38.97187133755819|(10,[0,2,3,4,7],[...|  1.0|
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| ba

In the output we can see the result of our transformation—a column called features that has our
previously raw data. What’s happening behind the scenes is actually pretty simple. RFormula
inspects our data during the fit call and outputs an object that will transform our data according
to the specified formula, which is called an RFormulaModel. This “trained” transformer always
has the word Model in the type signature. When we use this transformer, Spark automatically
converts our categorical variable to Doubles so that we can input it into a (yet to be specified)
machine learning model. In particular, it assigns a numerical value to each possible color
category, creates additional features for the interaction variables between colors and
value1/value2, and puts them all into a single vector. We then call transform on that object in
order to transform our input data into the expected output data.
Thus far you (pre)processed the data and added some features along the way. Now it is time to
actually train a model (or a set of models) on this dataset. In order to do this, you first need to
prepare a test set for evaluation. Let’s create a simple test set based off a random split of the data now (we’ll be using this test set
throughout the remainder of the chapter):



In [5]:
train, test = preparedDF.randomSplit([0.7, 0.3])

Estimators

Now that we have transformed our data into the correct format and created some valuable
features, it’s time to actually fit our model. In this case we will use a classification algorithm
called logistic regression. To create our classifier we instantiate an instance of
LogisticRegression, using the default configuration or hyperparameters. We then set the label
columns and the feature columns; the column names we are setting—label and features—are
actually the default labels for all estimators in Spark MLlib, and in later chapters we omit them:

In [6]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label",featuresCol="features")

Before we actually go about training this model, let’s inspect the parameters. This is also a great
way to remind yourself of the options available for each particular model:

In [8]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

While the output is too large to reproduce here, it shows an explanation of all of the parameters
for Spark’s implementation of logistic regression. The explainParams method exists on all
algorithms available in MLlib.
Upon instantiating an untrained algorithm, it becomes time to fit it to data. In this case, this
returns a LogisticRegressionModel:

In [9]:
fittedLR = lr.fit(train)

This code will kick off a Spark job to train the model. As opposed to the transformations that you
saw throughout the book, the fitting of a machine learning model is eager and performed
immediately.
Once complete, you can use the model to make predictions. Logically this means tranforming
features into labels. We make predictions with the transform method. For example, we can
transform our training dataset to see what labels our model assigned to the training data and how
those compare to the true outputs. This, again, is just another DataFrame we can manipulate.
Let’s perform that prediction with the following code snippet:


In [10]:
fittedLR.transform(train).select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



Our next step would be to manually evaluate this model and calculate performance metrics like
the true positive rate, false negative rate, and so on. We might then turn around and try a
different set of parameters to see if those perform better. However, while this is a useful process,
it can also be quite tedious. Spark helps you avoid manually trying different models and
evaluation criteria by allowing you to specify your workload as a declarative pipeline of work
that includes all your transformations as well as tuning your hyperparameters.

Pipelines

As you probably noticed, if you are performing a lot of transformations, writing all the steps and
keeping track of DataFrames ends up being quite tedious. That’s why Spark includes the
Pipeline concept. A pipeline allows you to set up a dataflow of the relevant transformations
that ends with an estimator that is automatically tuned according to your specifications, resulting
in a tuned model ready for use. Figure 24-4 illustrates this process.

As already mentioned, transformers help us manipulate our current columns in one way or
another. Manipulating these columns is often in pursuit of building features (that we will input
into our model). Transformers exist to either cut down the number of features, add more features,
manipulate current ones, or simply to help us format our data correctly. Transformers add new
columns to DataFrames.
When we use MLlib, all inputs to machine learning algorithms (with several exceptions
discussed in later chapters) in Spark must consist of type Double (for labels) and
Vector[Double] (for features). The current dataset does not meet that requirement and therefore
we need to transform it to the proper format.
To achieve this in our example, we are going to specify an RFormula. This is a declarative
language for specifying machine learning transformations and is simple to use once you
understand the syntax. RFormula supports a limited subset of the R operators that in practice
work quite well for simple models and manipulations (we demonstrate the manual approach to
this problem in Chapter 25). The basic RFormula operators are:


In [11]:
train, test = df.randomSplit([0.7, 0.3])

Now that you have a holdout set, let’s create the base stages in our pipeline. A stage simply
represents a transformer or an estimator. In our case, we will have two estimators. The RFomula
will first analyze our data to understand the types of input features and then transform them to
create new features. Subsequently, the LogisticRegression object is the algorithm that we will
train to produce a model:

In [12]:
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

We will set the potential values for the RFormula in the next section. Now instead of manually
using our transformations and then tuning our model we just make them stages in the overall
pipeline, as in the following code snippet:


We will set the potential values for the RFormula in the next section. Now instead of manually
using our transformations and then tuning our model we just make them stages in the overall
pipeline, as in the following code snippet:


In [13]:
from pyspark.ml import Pipeline
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

Training and evaluation

Now that you arranged the logical pipeline, the next step is training. In our case, we won’t train
just one model (like we did previously); we will train several variations of the model by
specifying different combinations of hyperparameters that we would like Spark to test. We will
then select the best model using an Evaluator that compares their predictions on our validation
data. We can test different hyperparameters in the entire pipeline, even in the RFormula that we
use to manipulate the raw data. This code shows how we go about doing that:


In [14]:
from pyspark.ml.tuning import ParamGridBuilder
params = ParamGridBuilder()\
  .addGrid(rForm.formula, [
    "lab ~ . + color:value1",
    "lab ~ . + color:value1 + color:value2"])\
  .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
  .addGrid(lr.regParam, [0.1, 2.0])\
  .build()

In our current paramter grid, there are three hyperparameters that will diverge from the defaults:
Two different versions of the RFormula
Three different options for the ElasticNet parameter
Two different options for the regularization parameter
This gives us a total of 12 different combinations of these parameters, which means we will be
training 12 different versions of logistic regression. We explain the ElasticNet parameter as
well as the regularization options in Chapter 26.
Now that the grid is built, it’s time to specify our evaluation process. The evaluator allows us to
automatically and objectively compare multiple models to the same evaluation metric. There are
evaluators for classification and regression, covered in later chapters, but in this case we will use
the BinaryClassificationEvaluator, which has a number of potential evaluation metrics, as
we’ll discuss in Chapter 26. In this case we will use areaUnderROC, which is the total area under
the receiver operating characteristic, a common measure of classification performance:

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()\
  .setMetricName("areaUnderROC")\
  .setRawPredictionCol("prediction")\
  .setLabelCol("label")

Now that we have a pipeline that specifies how our data should be transformed, we will perform
model selection to try out different hyperparameters in our logistic regression model and
measure success by comparing their performance using the areaUnderROC metric.
As we discussed, it is a best practice in machine learning to fit hyperparameters on a validation
set (instead of your test set) to prevent overfitting. For this reason, we cannot use our holdout test
set (that we created before) to tune these parameters. Luckily, Spark provides two options for
performing hyperparameter tuning automatically. We can use TrainValidationSplit, which
will simply perform an arbitrary random split of our data into two different groups, or
CrossValidator, which performs K-fold cross-validation by splitting the dataset into k nonoverlapping, randomly partitioned folds:

In [17]:
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit()\
  .setTrainRatio(0.75)\
  .setEstimatorParamMaps(params)\
  .setEstimator(pipeline)\
  .setEvaluator(evaluator)

Let’s run the entire pipeline we constructed. To review, running this pipeline will test out every
version of the model against the validation set. Note the type of tvsFitted is
TrainValidationSplitModel. Any time we fit a given model, it outputs a “model” type:

In [18]:
tvsFitted = tvs.fit(train)

And of course evaluate how it performs on the test set!

We can also see a training summary for some models. To do this we extract it from the pipeline,
cast it to the proper type, and print our results. The metrics available on each model are discussed
throughout the next several chapters. Here’s how we can see the results:


/ in Scala
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.LogisticRegressionModel
val trainedPipeline = tvsFitted.bestModel.asInstanceOf[PipelineModel]
val TrainedLR = trainedPipeline.stages(1).asInstanceOf[LogisticRegressionModel]
val summaryLR = TrainedLR.summary
summaryLR.objectiveHistory // 0.6751425885789243, 0.5543659647777687, 0.473776...
